In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dt = 1e-2;
mass = m = 1.;
charge = q = 1.;
vAc = 3e-4;

duration = 1000;

v = np.array([0., 1., 0.]);
x = np.array([-1., 0., 0.]);

B = np.array([0., 0., 1.]);
E = np.array([0., 0., 0.]);

X = np.zeros((duration,3)) 
V = np.zeros((duration,3)) 

### Runga Kutta 4th Order

In [ ]:
def rk4(x, v, a, dt):
    """
    Implements the fourth-order Runge-Kutta (RK4) method for particle pushing.

    Args:
        x (numpy.ndarray): Initial position of the particle (3D vector).
        v (numpy.ndarray): Initial velocity of the particle (3D vector).
        a (callable): Acceleration function that takes position and velocity as arguments.
        dt (float): Time step.

    Returns:
        tuple: Updated position and velocity of the particle after the push.
    """
    # Define the RK4 coefficients
    k1_v = a(x, v)
    k1_x = v

    k2_v = a(x + 0.5 * dt * k1_x, v + 0.5 * dt * k1_v)
    k2_x = v + 0.5 * dt * k1_v

    k3_v = a(x + 0.5 * dt * k2_x, v + 0.5 * dt * k2_v)
    k3_x = v + 0.5 * dt * k2_v

    k4_v = a(x + dt * k3_x, v + dt * k3_v)
    k4_x = v + dt * k3_v

    # Update position and velocity using RK4
    x_new = x + (dt / 6) * (k1_x + 2 * k2_x + 2 * k3_x + k4_x)
    v_new = v + (dt / 6) * (k1_v + 2 * k2_v + 2 * k3_v + k4_v)

    return x_new, v_new

### Leapfrog
https://www.particleincell.com/2011/velocity-integration/

In [ ]:
def EvalE(x):
    pass

In [ ]:
for time in range(duration):
    # move velocity back by 0.5dt
    E = EvalE(x);
    v = v - 0.5*charge/mass*E*dt;

    # main loop
    E = EvalE(x);
    v = v + charge/mass*E*dt;
    x = x + v*dt;

### Tajima Implicit
https://www.particleincell.com/2011/vxb-rotation/

In [ ]:
for time in range(duration):
    v_minus_half = v - 0.5*q/m*E*dt;
    B_mag = np.linalg.norm(B)
    eps = q*B_mag/m*dt/2 # = omega*dt/2
    R = 1/B_mag * [[0, B[2], -B[1]], [-B[2], 0, B[0]], [B[1], -B[0], 0]]
    M = np.eye(3) - R * eps
    M_inv = np.linalg.inv(M) # matrix inversion
    v = M_inv @ M @ v_minus_half + M_inv @ E * q/m * dt
    x = x + v*dt

### Tajima Explicit
https://www.particleincell.com/2011/vxb-rotation/

In [ ]:
for time in range(duration):
    v_minus_half = v - 0.5*q/m*E*dt;
    B_mag = np.linalg.norm(B)
    eps = q*B_mag/m*dt/2 # = omega*dt/2
    R = 1/B_mag * [[0, B[2], -B[1]], [-B[2], 0, B[0]], [B[1], -B[0], 0]]
    M = np.eye(3) - R * eps
    v = q/m*E*dt/2 + M @ (v_minus_half + E * q/m * dt / 2)
    x = x + v*dt

### Boris
https://www.particleincell.com/2011/vxb-rotation/

In [ ]:
for time in range(duration):
    t = charge / mass * B * 0.5 * dt;
    s = 2. * t / (1. + t*t);
    v_minus = v + charge / (mass * vAc) * E * 0.5 * dt;
    v_prime = v_minus + np.cross(v_minus,t);
    v_plus = v_minus + np.cross(v_prime,s);
    v = v_plus + charge / (mass * vAc) * E * 0.5 * dt;
    x += v * dt;
    X[time,:] = x;
    V[time,:] = v; 

In [ ]:
plt.figure(figsize=(5,5))

plt.plot(X[:,0],X[:,1],'k',linewidth=2.0); 
plt.xlabel(r'$x/d_{\rm p}$',fontsize=16)
plt.ylabel(r'$y/d_{\rm p}$',fontsize=16)

plt.show()